<a href="https://colab.research.google.com/github/LivroRelampagos/GLM_Brasil/blob/main/C%C3%B3digo_2_2_Gera_NetCDF_Mensal_glm_ELISA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GOES-16: REDE GLM (PARTE II.II)**

**Gera arquivo Netcdf Mensal do GLM**


---
**OBJETIVO:** Este Google Colab acumula os dados de flashes do GLM abordo do GOES-16 para escala **MENSAL**.

**DADOS DE ENTRADA**: Dados de flashes do GOES-16. 

**Dados:** Arquivos de relâmpagos a cada 5 min. **Local:** Os dados são processados e fornecidos pelo CPTEC/INPE e estão disponiveis [aqui](http://ftp.cptec.inpe.br/goes/goes16/goes16_web/glm_acumulado_nc/). 
**Nome:** S11635949_202011010000.nc.
**Formato:** Netcdf	
---
**DADOS DE SAÍDA:** Arquivo netcdf de flashes por **MÊS**.
---
**PROCEDIMENTO REALIZADO:** Os seguintes procedimentos são realizados nesse código:
1.   Instalação da biblioteca `pyunpack`
2.   Importação das bibliotecas
3.   Monta o drive da pessoa
4.   Download dos dados através do Drive
5.   Descompacta os dados baixados
6.   Acumula os flashes por mês
7.   Salva os dados mensais num arquivo netcdf
 
---
**OBSERVAÇÕES IMPORTANTES**: 
1. Mudar o caminho para seu drive. Veja em: **###MUDANÇAS AQUI####**
---
***Realizado por: [João G M Ribeiro](https://github.com/Joaogmr472/) e [Enrique V Mattos](https://github.com/evmpython) - 17/mar/2022*** 

# **###MUDANÇAS AQUI###**

- Insira o ano desejado

In [ ]:
ano = '2021'

- Insira o caminho do seu drive

In [ ]:
# local dos dados a cada 5 min
path_5min = '/content/drive/MyDrive/MESTRADO/DADOS/GLM/DADOS_5min/2021/'

# pasta de saída dos dados horários
path = '/content/drive/MyDrive/MESTRADO/DADOS/GLM/DADOS_mensal/'

# **Instala biblioteca que descompacta arquivos .rar**

In [ ]:
!pip install patool pyunpack

     |████████████████████████████████| 77 kB 2.5 MB/s 


# **Importa bibliotecas**

In [ ]:
import xarray as xr
import time as t
import glob
import numpy as np
from pyunpack import Archive
import os
import pandas as pd
from datetime import datetime
from tqdm.notebook import tqdm_notebook

import warnings
warnings.filterwarnings("ignore")

# **Acessando o drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

os.makedirs('input', exist_ok=True)

Mounted at /content/drive


# **Decompacta os arquivos baixados**

In [ ]:
# Looping para cada mês do ano
for i in range(1, 13):
  print(i)
  Archive(f'{path_5min}{str(i).zfill(2)}.rar').extractall(f'/content/input/') 

1
2
3
4
5
6
7
8
9
10
11
12


#**Acumula os dados de flashes: por Mês**

In [ ]:
# Inicia o contador de tempo
start_time = t.time()

# Data inicial
date_in = datetime(int(ano), 1, 1,0)
date_ini = date_in.strftime('%Y%m%d %H') 

# Data final
date_en = datetime(int(ano), 12, 31,23)
date_end = date_en.strftime('%Y%m%d %H')

# Número  de dias 
n_days = date_en - date_in

print(n_days)
print(date_in)
print(date_ini)

# Loop por mês
for file in  tqdm_notebook(pd.date_range(date_ini, date_end, freq='M')):
    
  # extrai a data
  ano = file.strftime('%Y')
  mes = file.strftime('%m')

  # monta lista de arquivos daquele mês
  files = glob.glob(f'/content/input/S11635949_{ano}{mes}*.nc')
    
  # Se existe arquivos naquele mês
  if len(files) > 0:
    # leitura dos dados num único dataset
    glm = xr.open_mfdataset(f'/content/input/S11635949_{ano}{mes}*.nc', parallel=True)

    # reamostra para escala mensal
    glm['flash'].resample(time='1M').sum().to_netcdf(f'{path}{ano}/flash_mensal_goes16_{ano}-{mes}.nc') 

# Contabiliza o tempo para baixar os dados
print('\nTempo de concatenar os dados:', (round((t.time() - start_time),2))/60., 'minutos.')

364 days, 23:00:00
2021-01-01 00:00:00
20210101 00


  0%|          | 0/12 [00:00<?, ?it/s]


Tempo de concatenar os dados: 248.50566666666666 minutos.
